In [1]:
%matplotlib widget

# Session 2

## First X-ray radiograph simulations with ![gVXR](img/gvxr_logo.png)

## Author: Franck Vidal

(version 1.0, 22 Sep 2022)

# Aims of this session

- Create our first X-ray simulation, step-by-step;
- Save our X-ray image in a file format that preserves the original dynamic range;
- Visualise the results with 3 different look-up tables;
- Visualise the 3D environment.

![](img/02-visualisation.png)

# Main steps

![](img/dragon2.jpg)

There are 6 main steps to simulate an X-ray image:

1. Create a renderer (OpenGL context)
2. Set the X-ray source
3. Set the Spectrum
4. Set the Detector
5. Set the Sample
6. Compute the corresponding X-ray image.

## Import packages

This is essentially just boilerplate code to set everything up for the session. We don't need to know the exact details at this stage so for now we can safely run this cell and move on. However, for the curious the comments explain what the various packages are for.  

In [3]:
import os
import numpy as np # Who does not use Numpy?

import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting
from matplotlib.colors import LogNorm # Look up table
from matplotlib.colors import PowerNorm # Look up table

font = {'family' : 'serif',
         'size'   : 10
       }
matplotlib.rc('font', **font)

# Uncomment the line below to use LaTeX fonts
# matplotlib.rc('text', usetex=True)

from tifffile import imread, imwrite # Write TIFF files

import base64 # Save the visualisation

from gvxrPython3 import gvxr # Simulate X-ray images
from gvxrPython3.utils import saveProjections # Plot the X-ray image in linear, log and power law scales
from gvxrPython3.utils import compareWithGroundTruth # Plot the ground truth, the test image, and the relative error map in %
from gvxrPython3.utils import interactPlotPowerLaw # Plot the X-ray image using a Power law look-up table
from gvxrPython3.utils import visualise # Visualise the 3D environment if k3D is supported

Speckpy is not install, you won't be able to load a beam spectrum using Speckpy
SimpleGVXR 2.0.2 (2022-09-23T13:26:05) [Compiler: GNU g++] on Linux
gVirtualXRay core library (gvxr) 2.0.2 (2022-09-23T13:26:05) [Compiler: GNU g++] on Linux


## Create an OpenGL context

Our first real step is to create what is known as an OpenGl context. This can be thought of a digital blank canvas onto which we will place our objects (Detector, Sample, X-ray source etc). Depending on your Operating system GVXR supports 2 different types of OpenGl contexts (backends), "OPENGL" and "EGL".

- "OPENGL" Creates an interactive window (available on Linux, MacOS, and Windows) for use on Laptop/desktop computers.
- "EGL" Creates a context without a window (available on Linux and MacOS, but not Windows) for use on supercomputers or the cloud.

For this tutorial we will stick with "OPENGL"

In [4]:
print("Create an OpenGL context")

window_id = 0
opengl_major_version = 4
opengl_minor_version = 5

gvxr.createOpenGLContext(window_id, opengl_major_version, opengl_minor_version);

backend = "OPENGL"
visible = True

# gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

visible = False
# gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

backend = "EGL"
# visible has no effect with EGL
# gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

Create an OpenGL context


Wed Sep 28 16:54:00 2022 ---- Create window (ID: 0)
Wed Sep 28 16:54:00 2022 ---- Initialise GLFW
Wed Sep 28 16:54:00 2022 ---- Create an OpenGL window with a 4.5 context.
Wed Sep 28 16:54:01 2022 ---- Make the window's context current
Wed Sep 28 16:54:01 2022 ---- Initialise GLEW
Wed Sep 28 16:54:01 2022 ---- OpenGL vendor: Intel
Wed Sep 28 16:54:01 2022 ---- OpenGL renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
Wed Sep 28 16:54:01 2022 ---- OpenGL version: 4.6 (Core Profile) Mesa 21.2.6
Wed Sep 28 16:54:01 2022 ---- Use OpenGL 4.5.
Wed Sep 28 16:54:01 2022 ---- Initialise the X-ray renderer if needed and if possible


## Setting up the X-ray source

Next we need to set the position of the X-ray source. For this we use the `gvxr.setSourcePosition(x,y,z,units)` function. it takes in 4 values. The x,y and z coordinates as 3 floating point (decimal) values and a string (literal letters or numbers surrounded by quotes) to denote the units ("mm","cm","m" etc ...).

In the next code cell set the X-ray Source to be at position x = 0.0 cm, y = -40.0 cm and z = 0.0 cm.

We also need to define a beam shape. GVXR allows for two choices
- Cone beam: `gvxr.usePointSource()`
- Parallel beam (e.g. synchrotron): `gvxr.useParallelBeam()`

For now in the next cell set the beam shape to be a cone beam.

Finally we need to set the beam spectrum. Out of the box GVXR supports Monochromatic and PolyChromatic sources. You can also use other external packages (e.g. Spekpy) to generate more realistic/complex spectra. This will be covered in a later session. For now we will stick with a simple Monochromatic source. This can be set with the function `gvxr.setMonoChromatic(energy,unit_of_energy,number_of_photons)`. This takes in three values a float for the beam energy. A string to denote the energy units (can be any of "eV", "keV" or "MeV" take care with capitalisation) and an integer (whole number) for the number of photons.

 In the next cell use the setMonoChromatic function to define a monochromatic X-ray source of energy 0.08 MeV (i.e. 80 keV) with 1000 photons per ray.

## Setting up the detector:

In GVXR the X-ray detector is defined as a 2D plane of evenly spaced pixels. Thus we need to specify 4 things:

- The resolution (number of pixels in x and y)
- The Pixel spacing (this defines the physical size of the detector)
- The position of the top left corner
- It's orientation in 3D space

The resolution is set with `gvxr.setDetectorNumberOfPixels(Px,Py)` where Px and Py are integers defining the number of pixels in x and y respectively. Note: this is also the resolution of the final output image.

The pixel spacing is set with `gvxr.setDetectorPixelSize(width,height,units)` where width and height are floating point numbers defining the physical distance between pixels and units is once again a string to denote the units ("mm","cm","m" etc ...). Note: The Pixel spacing and resolution together define the physical size of the detector. For example a detector with with 600 pixels in x and 300 in y with a spacing of 0.1 mm in both directions would have a physical size of 60 mm by 30 mm.

The position of the X-ray detector is taken from the top-left corner, which is just a convention in computer graphics. It is defined in much the same way as for the X-ray source. Only we use the `gvxr.setDetectorPosition(x,y,z,units)` function. Much like the source case this function takes in 4 values. The x,y and z coordinates as 3 floating point values and a string to denote the units ("mm","cm","m" etc ...).

Finally since the detector is just a 2D plane we also need to set its orientation in 3D space. To do this we need to use `gvxr.setDetectorUpVector(ix,jy,kz)`. Here, the three inputs ix, jy and kz represent a unit vector, the direction of which points along the Y-axis of our detector. Note: By convention we generally define this to point along an axis in 3D space. 

With all this in mind. In the next cell set the top left corner of the detector to be at `x = 0.0 cm`, `y = 10.0 cm` and `z = 0.0 cm`. Also set the detector resolution to 640 by 320 pixels, spaced 0.5 mm apart in both x and y. Finally, set the Y-axis of the detector along the unit vector `(0,0,-1)`. That is pointing along the negative Z-axis.

# Set the sample

- Welsh dragon in a STL file:
    - ID: "Dragon",
    - fname: "input_data/welsh-dragon-small.stl",
    - Unit: mm.
    
```python
gvxr.loadMeshFile(
    ID, # string
    fname, # string
    unit of length # string, e.g. "mm"
)
```    

Wed Sep 28 16:57:44 2022 ---- file_name:	input_data/welsh-dragon-small.stl	nb_faces:	457345	nb_vertices:	1372035	bounding_box (in cm):	(-4.47065, -74.9368, 23.5909)	(2.37482, -59.4256, 36.0343)


- Rotate the sample by 90 degrees around the axis (0, 0, 1)
    - ID: "Dragon",
    - angle: 90 degrees
    - x: 0,
    - y: 0, 
    - z: 1.
   
```python
gvxr.rotateNode(
    ID, # string
    90, # float
    0, # float
    0, # float
    1 # float
)
```

# Move the sample to the centre of the world

No ID is provided in case the sample is made of several components.

```python
gvxr.moveToCentre()
```    

or

```python
gvxr.moveToCenter()
```    

or if you prefer the American spelling

# Set Material of the sample:

We will cover this in more detail in the next section but for now we will keep things simple and set the sample to be made from a single element. 

- For a chemical element such as iron, you can use the name, atomic (Z) number or symbol:
    - `gvxr.setElement(ID, "Iron")`
    - `gvxr.setElement(ID, 26)`, or
    - `gvxr.setElement(ID, "Fe")`

For now set the dragon model to be made entirly from Titanium (Hint: the Z number is 22).

# Compute the corresponding X-ray image

Get a 2D array

```python
xray_image = gvxr.computeXRayImage()
```

or make sure it's a Numpy array in float32 with

```python
xray_image = np.array(gvxr.computeXRayImage()).astype(np.single)
```

# Update the visualisation window

To update the geometry and render a static view you can use:
- gvxr.displayScene()

or if you would like an interactive view where you can pan, rotate, zoom ect you can use:
- gvxr.renderLoop()

**Note: thease functions only work for "OPENGL" and have no effect when using "EGL" on supercomputers or the cloud, as there is no window to render to. They will however, work on a desktop or laptop computer.**

When running in an interactive loop you can rotate the 3D scene and zoom-in with the mouse buttons and scroll wheel.

Useful Keys are:
- Q/Escape: to quit the event loop\n'
- B: display/hide the X-ray beam\n'
- W: display the polygon meshes in solid or wireframe\n'
- N: display the X-ray image in negative or positive\n'
- H: display/hide the X-ray detector
        

In [ ]:
gvxr.displayScene()

# Create the output directory if needed

In [ ]:
if not os.path.exists("output_data"):
    os.mkdir("output_data")

# Save the X-ray image in a TIFF file

This stores the data using single-precision floating-point numbers. You can do this using gVXR directly with:

```python
gvxr.saveLastXRayImage(
    filename # string
)
```
Or you can use the the tifffile package
```python
from tifffile import imwrite
imwrite(
    filename, # string
    xray_image # 2D array
)
```
For now save the X-ray as a tiff stack with the filename 'output_data/02-gvxr-save.tif'

# Display the X-ray image

### using a linear colour scale

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Image simulated using gVirtualXray\nusing a linear colour scale")
plt.imshow(xray_image, cmap="gray")
plt.colorbar(orientation='vertical');
plt.margins(0,0)

### using a logarithmic colour scale

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Image simulated using gVirtualXray\nusing a logarithmic colour scale")
plt.imshow(xray_image, cmap="gray", norm=LogNorm(vmin=xray_image.min(), vmax=xray_image.max()))
plt.colorbar(orientation='vertical');
plt.margins(0,0)

### Using a Power-law colour scale

In [ ]:
interactPlotPowerLaw(xray_image, gamma=1.5, figsize=(10, 5))

# Display the X-ray image and compare three different lookup tables

Replace `???` below with the value of gamma that you selected above.

In [ ]:
saveProjections(xray_image, "output_data/02-projections-dragon-TiAlV.pdf", gamma=???, figsize=(12.5, 5))

# Get some image statistics using Numpy

and compare with the values I got. We should get the same, or at least something comparable.

| What? | Value (in keV) |
|-------|-------|
| Min pixel value: | 0.0056294748 |
| Mean pixel value: | 56.367035 |
| Median pixel value: | 80.0 |
| Stddev pixel value: | 33.96409 |
| Max pixel value: | 80.0 |

In [ ]:
print("Min pixel value:", np.min(xray_image))
print("Mean pixel value:", np.mean(xray_image))
print("Median pixel value:", np.median(xray_image))
print("Stddev pixel value:", np.std(xray_image))
print("Max pixel value:", np.max(xray_image))

# Compare with the ground truth

In [ ]:
ground_truth = imread("input_data/02-dragon-TiAlV-groundtruth.tif")
compareWithGroundTruth(ground_truth, xray_image, figsize=(12.5, 5))

# Change the sample's colour

By default the object is white, which is not always pretty. Let's change it to purple.

In [ ]:
red = 102 / 255
green = 51 / 255
blue = 153 / 255
gvxr.setColour("Dragon", red, green, blue, 1.0)

# 3D visualisation using k3D if possible

In [ ]:
plot=visualise(use_log=True)
plot.display()

In [ ]:
if plot is not None:
    plot.fetch_screenshot()

    data = base64.b64decode(plot.screenshot)
    with open("output_data/02-visualisation.png",'wb') as fp:
        fp.write(data)

# Change the background colour to white

This image can be used in a research paper to illustrate the simulation environment.

In [ ]:
gvxr.setWindowBackGroundColour(1.0, 1.0, 1.0)

# Update the visualisation window

In [ ]:
gvxr.displayScene()

# Take the screenshot and save it in a file

In [ ]:
screenshot = gvxr.takeScreenshot()
plt.imsave("output_data/02-screenshot.png", np.array(screenshot))

# or display it using Matplotlib

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(screenshot)
plt.title("Screenshot of the X-ray simulation environment")
plt.axis('off');

# Interactive visualisation

The user can rotate the 3D scene and zoom-in and -out in the visualisation window.

- Keys are:
    - Q/Escape: to quit the event loop (does not close the window)
    - B: display/hide the X-ray beam
    - W: display the polygon meshes in solid or wireframe
    - N: display the X-ray image in negative or positive
    - H: display/hide the X-ray detector
- Mouse interactions:
    - Zoom in/out: mouse wheel
    - Rotation: Right mouse button down + move cursor
    
**Note: this function has no effect on supercomputers and on the cloud, but will work on a desktop or laptop computer.**

In [ ]:
gvxr.renderLoop()

# All done

In [ ]:
gvxr.terminate()